Copyright (C) Egon Kidmose 2015-2017

This file is part of lstm-rnn-correlation.

lstm-rnn-correlation is free software: you can redistribute it and/or
modify it under the terms of the GNU Lesser General Public License as
published by the Free Software Foundation, either version 3 of the
License, or (at your option) any later version.

lstm-rnn-correlation is distributed in the hope that it will be
useful, but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
Lesser General Public License for more details.

You should have received a copy of the GNU Lesser General Public
License along with lstm-rnn-correlation. If not, see
<http://www.gnu.org/licenses/>.


# Alert correlation with a Long Short-Term Memoroy (LSTM) Recurrent Neural Network(RNN) and cosine similarity

**Author:** Egon Kidmose, egk@es.aau.dk

In network security a common task is to detect network intrusions and for this purpose an Intrusion Detections System (IDS) can be used to raise alerts on suspicious network traffic.
Snort, Suricata and Bro are examples of free and open source IDSs (Commercial products also exist).
The alerts generally provides low level information such as recognition of strings that are known to be part of security exploits or anomalous connection rates for a host.
By grouping alerts that are correlated into higher level events, false positives might be suppressed and attack scenarios becomes easier to recognise. 
This is a take on how to correlate IDS alerts to determine which belong in the same group.

Alerts can be represented as log lines with various information such as time stamp, IP adresses, protocol information and a description of what triggered the alert. 
It is assumed that such a log lines hold the information needed to determine if two alerts are correlated or not.

The input to the neural network will be two alerts and the output will indicate if they are correlated or not. 
In further detail the inputs is two strings of ASCII characters of variable length. 
For the output a Cosine Similarity layer is implemented and used to produce an output in the range [-1,1], with -1 meaning opposite, 0 meaning orthogonal and 1 meaning the same. 

For the hidden layers only a single layers of Long Short-Term Memory (LSTM) cells is used.
It is an option to experiment with adding more.
Being reccurrent, such a layer handles variable length input well. 

While it turned out to be to challenging to implement, the initial idea was to let the two inputs pass through LSTM layers with identical weights.
The intent was to have them act as transformations into a space where cosine similarity could be used to measure similarity of the alerts.
However I have not succeded at tying the weights together.
As an alternative this might be achieved by using all training pairs in both original and swapped order.
The intuition is that this leads to two identical layers, but intuition also suggest that this is highly ineffective.

                      Output
                        |
    Cosine similarity   #
                       / \
        LSTM layers   #   #
                      |   |
        "alert number 1"  |
            "alert number 2"


Reference: Huang, Po-Sen, et al. "Learning deep structured semantic models for web search using clickthrough data." Proceedings of the 22nd ACM international conference on Conference on information & knowledge management. ACM, 2013.


In [ ]:
from __future__ import print_function

import sys
import os
import time

import numpy as np
import theano
import theano.tensor as T

import lasagne
from lasagne.layers import *
from lasagne.nonlinearities import *
from lasagne.updates import *
from lasagne.objectives import *

In [ ]:
def load_dataset(
    ntrain=100,
    nval=100,
    ntest=100,
):
    X_unit = ['abcdef', 'abcdef', 'qwerty']
    X_unit = [[ord(c) for c in w] for w in X_unit]
    X_unit = np.array(X_unit, dtype='int8')
    
    X_train = np.array(list(zip(X_unit, X_unit[::-1])))
    y_train = np.array([0, 1, 0])

    X_val, y_val = X_train, y_train
    X_test, y_test = X_train, y_train
    return X_train, y_train, X_val, y_val, X_test, y_test

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()

# For unit testing
# X_unit, y_unit, _, _, _, _ = load_dataset(2, min_digits=1, max_digits=1)
X_unit = ['abcdef', 'abcdef', 'qwerty']
X_unit = [[ord(c) for c in w] for w in X_unit]
X_unit = np.array(X_unit, dtype='int8')
print(X_unit)
n_alerts_unit, l_alerts_unit = X_unit.shape

# First line of network

In [ ]:
input_var = T.imatrix('inputs')
target_var = T.dvector('targets')

# Input layer
n_alerts = None
l_alerts = None
l_in = InputLayer(shape=(n_alerts, l_alerts), input_var=input_var, name='INPUT-LAYER') # 

# Test
pred_unit = get_output(l_in, inputs={l_in: input_var}).eval(
    {input_var: X_unit})
assert (pred_unit == X_unit).all(), "Unexpected output"

print(pred_unit)

In [ ]:
# Embedding layer
n_alphabet = 2**7 # All ASCII chars

l_emb = EmbeddingLayer(l_in, n_alphabet, n_alphabet, 
                         W=np.eye(n_alphabet),
                         name='EMBEDDING-LAYER')
l_emb.params[l_emb.W].remove('trainable') # Fix weight

# Test
pred_unit = get_output(l_emb, inputs={l_in: input_var}).eval(
    {input_var: X_unit})
assert (np.argmax(pred_unit, axis=2) == X_unit).all()
assert np.all(pred_unit.shape == (n_alerts_unit, l_alerts_unit, n_alphabet ))
print(pred_unit.shape)
print(pred_unit)


In [ ]:
# Recurrent LSTM layer
num_units = 10
l_lstm = LSTMLayer(l_emb, num_units=num_units, name='LSTM-LAYER')

# Test
pred_unit = get_output(l_lstm, inputs={l_in: input_var}).eval({input_var: X_unit})
assert pred_unit.shape == (n_alerts_unit, l_alerts_unit, num_units), "Unexpected dimensions"
print(pred_unit)

pred_unit = get_output(l_lstm, inputs={l_in: input_var}).eval({input_var: [[1],[1]]})
assert np.all(pred_unit[0] == pred_unit[1]), "Repeated alerts must produce the same"

pred_unit = get_output(l_lstm, inputs={l_in: input_var}).eval({input_var: [[1,1],[1,1]]})
assert np.all(pred_unit[0] == pred_unit[1]), "Repeated alerts must produce the same"

pred_unit = get_output(l_lstm, inputs={l_in: input_var}).eval({input_var: [[1,1],[0,1]]})
assert np.all(pred_unit[0] != pred_unit[1]), "Earlier must affect laters"

pred_unit = get_output(l_lstm, inputs={l_in: input_var}).eval({input_var: [[1,0],[1,1]]})
assert np.all(pred_unit[0,0] == pred_unit[1,0]), "Later must not affect earlier"
assert np.all(pred_unit[0,1] != pred_unit[1,1]), "Current must make a difference"

In [ ]:
# Slice Layer
# Pick the outputs for the last entry in the sequences. 

l_slice = SliceLayer(l_lstm, indices=-1, axis=1, name="SLICE-LAYER")

# Test
pred_unit = get_output(l_slice, inputs={l_in: input_var}).eval({input_var: X_unit})
assert pred_unit.shape == (n_alerts_unit, num_units), "Unexpected shape"

pred_unit_lstm = get_output(l_lstm, inputs={l_in: input_var}).eval({input_var: X_unit})
assert np.all(pred_unit_lstm[:, -1, :] == pred_unit), "Unexpected result of slicing"
print(pred_unit)

net = l_slice

# Clone line, tie weights

In [ ]:
# Create an identical test network, with tied weights
input_var2 = T.imatrix('inputs2')
l_in2 = InputLayer(
    shape=l_in.shape, 
    input_var=input_var2, 
    name=l_in.name+'2',
)
net2 = l_in2

for l in get_all_layers(net):
    print("{} ({}):".format(l.name, l))
    if type(l) == InputLayer:
        print(' - skipping')
        continue
    if type(l) == DenseLayer:
        net2 = DenseLayer(
            net2,
            num_units=l.num_units,
            W=l.W,
            b=l.b,
            nonlinearity=l.nonlinearity,
            name=l.name+'2',
        )
    elif type(l) == EmbeddingLayer:
        net2 = EmbeddingLayer(
            net2,
            l.input_size,
            l.output_size,
            W=l.W,
            name=l.name+'2',
        )
    elif type(l) == LSTMLayer:
        net2 = LSTMLayer(
            net2,
            l.num_units,
            ingate=Gate(W_in=l.W_in_to_ingate, W_hid=l.W_hid_to_ingate, W_cell=l.W_cell_to_ingate, b=l.b_ingate, nonlinearity=l.nonlinearity_ingate),
            forgetgate=Gate(W_in=l.W_in_to_forgetgate, W_hid=l.W_hid_to_forgetgate, W_cell=l.W_cell_to_forgetgate, b=l.b_forgetgate, nonlinearity=l.nonlinearity_forgetgate),
            cell=Gate(W_in=l.W_in_to_cell, W_hid=l.W_hid_to_cell, W_cell=None, b=l.b_cell, nonlinearity=l.nonlinearity_cell),
            outgate=Gate(W_in=l.W_in_to_outgate, W_hid=l.W_hid_to_outgate, W_cell=l.W_cell_to_outgate, b=l.b_outgate, nonlinearity=l.nonlinearity_outgate),
            nonlinearity=l.nonlinearity,
            cell_init=l.cell_init,
            hid_init=l.hid_init,
            backwards=l.backwards,
            learn_init=l.learn_init,
            peepholes=l.peepholes,
            gradient_steps=l.gradient_steps,
            grad_clipping=l.grad_clipping,
            unroll_scan=l.unroll_scan,
            precompute_input=l.precompute_input,
            # mask_input=l.mask_input, # AttributeError: 'LSTMLayer' object has no attribute 'mask_input'
            name=l.name+'2',
        )
    elif type(l) == SliceLayer:
        net2 = SliceLayer(
            net2,
            indices=l.slice,
            axis=l.axis,
            name=l.name+'2',
        )
    else:
        raise ValueError("Unhandled layer: {}".format(l))
    print(' - added layer: {} ({})'.format(get_all_layers(net2)[-1], get_all_layers(net2)[-1].name))
print()

# Test
pred_unit = get_output(net, inputs={l_in: input_var}).eval({input_var: X_unit})
pred_unit2 = get_output(net2, inputs={l_in2: input_var2}).eval({input_var2: X_unit})
assert pred_unit.shape == pred_unit2.shape
assert np.all(pred_unit == pred_unit2)

print(pred_unit)
print(pred_unit2)

$\cos(\theta) = {A \cdot B \over \|A\| \|B\|} = \frac{ \sum\limits_{i=1}^{n}{A_i \times B_i} }{ \sqrt{\sum\limits_{i=1}^{n}{(A_i)^2}} \times \sqrt{\sum\limits_{i=1}^{n}{(B_i)^2}} }$

In [ ]:
# Cosine similarity layer implementation
class CosineSimilarityLayer(MergeLayer):
    """Calculates the cosine of two inputs."""
    def __init__(self, incoming1, incoming2, **kwargs):
        """Instantiates the layer with incoming1 and incoming2 as the inputs."""
        incomings = [incoming1, incoming2]
        
        for incoming in incomings:
            if isinstance(incoming, tuple):
                if len(incoming) != 2:
                    raise NotImplementedError("Requires shape to be exactly (BATCH_SIZE, N).")
            elif len(incoming.output_shape) != 2:
                raise NotImplementedError("Requires shape to be exactly (BATCH_SIZE, N).")
                
        super(CosineSimilarityLayer, self).__init__(incomings, **kwargs)
    
    def get_output_shape_for(self, input_shapes):
        """Return output shape: (batch_size, 1)."""
        if len(input_shapes) != 2:
            raise ValueError("Requires exactly 2 input_shapes")

        for input_shape in input_shapes:
            if len(input_shape) != 2:
                raise NotImplementedError("Requires shape to be exactly (BATCH_SIZE, N).")

        return (input_shape[0],)
    
    def get_output_for(self, inputs, **kwargs):
        """Calculates the cosine similarity."""
        nominator = (inputs[0] * inputs[1]).sum(axis=1)
        denominator = T.sqrt((inputs[0]**2).sum(axis=1)) * T.sqrt((inputs[1]**2).sum(axis=1))
        return nominator/denominator
        
# Test
test_in_1 = InputLayer((None, None))
test_in_2 = InputLayer((None, None))
test_layer = CosineSimilarityLayer(test_in_1, test_in_2)
in1, in2 = T.dmatrices('in1', 'in2')

pred_unit = lasagne.layers.get_output(test_layer, inputs={
        test_in_1: in1,
        test_in_2: in2
    }).eval({
        in1: [[0, 1], [1, 0], [0, -1]],
        in2: [[0, 1], [0, 1], [0, 1]],
    })
assert len(pred_unit.shape) == len(test_layer.output_shape), "Dimension mismatch"
assert (pred_unit == [ 1.,  0., -1.]).all(), "Invalid output"
print('OK')

In [ ]:
# Cosine similarity layer
net = CosineSimilarityLayer(net, net2, name="COSINE-SIMILARITY-LAYER")

pred_unit = get_output(net, inputs={
        l_in: input_var, l_in2: input_var2
    }).eval({
        input_var: X_unit, input_var2: X_unit
    })

assert pred_unit.shape == (n_alerts_unit,)
print(pred_unit)


In [ ]:
# Sigmoid layer
net = NonlinearityLayer(net, nonlinearity=sigmoid)


In [ ]:
# Training
prediction = get_output(net)
loss = binary_crossentropy(prediction, target_var)
loss = loss.mean()
params = get_all_params(net, trainable=True)
updates = sgd(loss, params, learning_rate=0.1)

# Testing
test_prediction = get_output(net, deterministic=True)
test_loss = binary_crossentropy(test_prediction, target_var)
test_loss = test_loss.mean()
test_acc = T.mean(T.eq(test_prediction > 0.5, target_var),
                  dtype=theano.config.floatX)

train_fn = theano.function([input_var, input_var2, target_var], loss, updates=updates)
val_fn = theano.function([input_var, input_var2, target_var], [test_loss, test_acc])

In [ ]:
print("Starting training...")
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset(100,100,100)

num_epochs = 100
for epoch in range(num_epochs):
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, 3, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs[:,0], inputs[:,1], targets)
        train_batches += 1

    if (epoch+1) % 20 == 0:
        val_err = 0
        val_acc = 0
        val_batches = 0
        for batch in iterate_minibatches(X_val, y_val, 3, shuffle=False):
            inputs, targets = batch
            err, acc = val_fn(inputs[:,0], inputs[:,1], targets)
            val_err += err
            val_acc += acc
            val_batches += 1

        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        print("  validation accuracy:\t\t{:.2f} %".format(
            val_acc / val_batches * 100))

test_err = 0
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 3, shuffle=False):
    inputs, targets = batch
    err, acc = val_fn(inputs[:,0], inputs[:,1], targets)
    test_err += err
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))